#1 Leiðarval hjá Strætó
Í þessu verkefni á að búa til leitarvél fyrir Strætó, notast verður við raungögn frá Strætó
sem er að finna á
https://straeto.is/um-straeto/opin-rafraen-gogn.
Til einföldunar munuð þið nota skrána gtfs.zip sem er að finna á Canvas síðu námskeiðsins til að tryggja að
allir noti sömu útgáfu.


## 2.1 Inntak
Inntakið er tímatafla strætó, listi yfir leiðir, vagna og stoppistöðvar. Við skilgreinum netið
G á eftirfarandi hátt. Hver einasta stoppistöð verður hnútur.
Ef vagn ekur á milli stoppistöðva x og y á tímum t1 og t2 með engum öðrum stoppum á
milli þá búum við til nýjan hnút v með leggina (x, v), (v, y) og merkjum þá með tímunum t_1 og t_2. Hnúturinn v geymir upplýsingar um hvaða vagni sá hnútur tilheyrir, athugið þessi
hnútur verður ekki endurnýttur.
Fyrir leiðina sem er sýnd á myndinni þá samsvarar hún fimm hnútum. Hnútar 1, 3, 5
eru stoppistöðvarnar Hlemmur B, Sólfarið og Harpa. Hnútur 2 tengir saman Hlemm B og
Sólfarið og leggirnir hafa tímann 10:22 og 10:25. Hnútur 4 tengir saman Sólfarið og Hörpu,
fyrri leggurinn hefur tímann 10:25 (ekki sýnilegt á mynd) og sá seinni 10:26. Þetta er dæmi
um löglegan veg þar sem allir leggir samsvara vögnum sem við getum tekið og tímarnir
stemma, þ.e. eru í vaxandi röð

## 2.2 Reiknirit
Einfaldasta tilfellið sem við viljum leysa þá fá um við par af stoppistöðvum og tíma, (x, y, t)
og úttakið er leið í strætókerfinu frá x til y sem fer frá x á tíma t′ ≥ t og lágmarkar
komutímann á áfangastað. Ef við skiptum um strætisvagn þá samsvarar það því að "bíða"á
hnútnum sem er á stoppistöð og taka legg sem tilheyrir öðrum vagni heldur en þeim sem
við komum með.
Til að leysa þetta verkefni er hægt að nota reiknirit Dijkstra með einni breytingu, í stað
þess að slaka (RELAX aðferðin) á leggjum með því að leggja saman vigtir þá notum við
komutímann á leggnum ef leggurinn er löglegur, þ.e. ef (u, v) er leggurinn sem við skoðum
með tímann t þá setjum við d[v] = min(d[v], t) ef t ≥ d[u].


# 2.3 Verkþættir
## 2.3.1 Þáttun (⋆)
Gögnin fyrir strætó eru gefin á GTFS formi, lýsingu á GTFS er að finna á https://gtfs.
org/schedule/reference/. Til að búa til netið þarf að þátta (e. parse) gögnin, þ.e. lesa
þau inn og tengja saman á skynsamlegan hátt.

Til einföldunar ætlum við eingöngu að nota eftirfarandi hluta af gögnunum

agency_id=1 eingöngu leiðir sem tilheyra Strætó BS merktar ST.*

Leiðirnar eru ólíkar milli vikudaga, gefum okkur að við séum að vinna með virka daga
og sleppum næturstrætó (leiðir 101-106). Það má líka sleppa helgidögum, þ.e. sem er
skilgreint í calendar_dates.csv skráni.

Sumar leiðir eru í pöntunarþjónustu, við höldum eingöngu þeim leggjum sem eru með
pickup_type=0 í stop_times.csv.

Búið til net sem samsvarar leiðarkerfinu á virkum degi. Sýnið fjölda hnúta og leggja í
netinu og finnið þá stoppistöð sem hefur hæstu útgráðuna (hvaða stöð og hver er útgráðan).
Netið sem þið búið til má vera hvernig sem er svo lengi sem það sé "pythonskt"í laginu.
Þ.e. við getum ítrað yfir hnúta í netinu og fyrir hvern hnút getum við ítrað yfir granna þess
hnúts. Útfærslan í Java í Algorithms bókinni uppfyllir þessi skilyrði og eftirfarandi einfaldi
python kóði gerir það líka.

```
G = {1: [2,3], 2: [], 3: [1,2] }
for v in G:
  print(u,':')
    for u in G[u,v]:
      print(u,' -> ',v)
```
Ef við viljum bæta við einhverjum upplýsingum þá er hægt að gera það í uppflettitöflu.
Ekki er leyfilegt að nota pakka fyrir net í python á borð við networkx eða sambærilegt.

In [72]:
import pandas as pd

# Lesa inn gögnin
stops_df = pd.read_csv('content/stops.txt')
trips_df = pd.read_csv('content/trips.txt')
stop_times_df = pd.read_csv('content/stop_times.txt')
routes_df = pd.read_csv('content//routes.txt', comment='#')
calendar_dates_df = pd.read_csv('content/calendar_dates.txt')
# næturstrætó
night_buses = ['101', '102', '103', '104', '105', '106']

# Filter -> bara sem byrjar á ST. & agency_id = 1 & er ekki næturstrætó
routes_df = routes_df[(routes_df['route_id'].str.startswith("ST.", na=False)) &
                      (routes_df['agency_id'] == 1) &
                      (~routes_df['route_short_name'].isin(night_buses))]

# Bæta við dálki sem hefur aðeins upplýsingar um vikudaga.
trips_df['day_pattern'] = trips_df['service_id'].apply(lambda x: x.split('_')[1])

# Filter -> bara virkir dagar.
def all_weekdays_no_weekends(pattern):
    weekdays = 'MTWTF'
    return all(day in pattern for day in weekdays if day != '-')
    
filtered_trips_df = trips_df[trips_df['day_pattern'].apply(all_weekdays_no_weekends)]

# Filter -> pickup_type = 0
filtered_stop_times_df = stop_times_df[stop_times_df['pickup_type'] == 0]

# Merge -> filtered_stop_times & stops til að fá nafn á stoppi
stop_times_with_stops_df = pd.merge(filtered_stop_times_df, stops_df, on='stop_id')

# Merge -> stop_times_with_stops & trips til að fá upplýsingar um ferðir
stop_times_with_stops_and_trips_df = pd.merge(stop_times_with_stops_df, filtered_trips_df, on='trip_id')

# Merge -> Henda þessu í öllu í eina klessu
stop_times_with_stops_trips_and_routes_df = pd.merge(stop_times_with_stops_and_trips_df, routes_df[['route_id', 'route_short_name']], on='route_id')

# Merge -> stop_times_with_stops_trips_and_routes & calendar_dates til að fá upplýsingar um dagsetningar
filtered_final_df = pd.merge(stop_times_with_stops_trips_and_routes_df, calendar_dates_df, on='service_id')

# Filter -> bara fyrir daginn 20240411
filtered_final_current_df = filtered_final_df[filtered_final_df['date'] == 20240411]


# 2.3.2 Leit (⋆⋆)
Útfærið leitarreikniritið á netinu eins og lýst var að ofan. Sýnið niðurstöðu leitarinnar þegar
við viljum komast frá Meistaravöllum (ath. það eru tvær stöðvar með þessu nafni) til FB,
kl 11:30.


## 2.3.3 Framsetning (⋆)
Takið löglega leið og búið til leiðarlýsingu, þ.e. hvaða vagn á að taka klukkan hvað og hvar
á að skipta um stöð, svipað og gert er á myndinni að ofan.

In [ ]:

# class Node:  # Hnutur
#     def __init__(self, name: str):
#         self.name = name  # Name of the stop

# class Edge:  # Leggur
#     def __init__(self, start: Node, end: Node, start_time: str, end_time: str):
#         self.start = start  # Starting stop
#         self.end = end  # Ending stop
#         self.start_time = start_time  # Departure time from the start stop
#         self.end_time = end_time  # Arrival time at the end stop


## 2.3.4 Labb (⋆⋆)
Sumar stoppistöðvar eru nálægt hvor annari en ekki hægt að taka strætó á milli þeirra. Breytið netinu og reikniritinu ykkar til að höndla þetta tilfelli, notið GPS hnitin sem eru gefin í gögnunum og veljið hæfilega fjarlægð sem þið eruð til í að láta fólk labba. T.d. er hægt að labba úr 13 í Suðurveri yfir í 4 á Kringlumýrarbraut en 13 stoppar ekki þar. Sýnið dæmi um leið þar sem komutími styttist við að leyfa labb. Þessi aðferð við að tengja saman stöðvar er ekki fullkomin og sýnir að oft er mikilvægara að hafa aðgang að betri gögnum, t.d. eru stoppistöðvar nálægt í planinu en taka langan tíma að labba á milli.

## 2.3.5 Tímamælingar (⋆)
Mælið fjölda fyrirspurna sem aðferðin ykkar getur svarað á sekúndu. Tryggið að mælingarnar blandi stuttum og löngum leiðum með ólíkum tímum.

## 2.3.6 Bestun og gagnagrindur (⋆⋆)
Notið niðurstöður úr tímamælingu til að ná betri afköstum. Hér skiptir máli hvernig framsetningin á netinu er framkvæmd, hvaða gagnagrindur eru notaðar og hvort hægt sé að bæta eitthvað.

## 2.3.7 Færri skiptingar (⋆⋆)
Í einhverjum tilfellum eru margar ólíkar leiðir sem taka jafnlangan tíma, þ.e. seinasti vagninn kemur á áfangastað á sama tíma en fjöldi vagna sem er tekinn er ekki sá sami. Útskýrið hvernig er hægt að breyta reikniritinu til skipta sem sjaldnast um vagn og útfærið þetta á netinu. Sýnið dæmi um strætóferð þar sem niðurstaðan breytist.

## 2.3.8 A(⋆⋆⋆)
Útfærið A∗ reikniritið (sjá kafla 10.8 í https://people.mpi-inf.mpg.de/~mehlhorn/ftp/
NewToolbox/spath.pdf) með því að velja heppilegt fjarlægðarfall f . Sýnið að fjarlægðar-
fallið sem er valið sé löglegt, þ.e. reikniritið skilar alltaf réttum niðurstöðum, og framkvæmið
tímamælingar sem mæla hversu miklu þessi breyting skilar.

## 2.3.9 Pokémon (⋆ ⋆ ⋆)
Finnið þá leið sem tekur mestan fjölda vagna. Athugið að þetta verkefni er mögulega NP-erfitt, takið eftir að netið eins og það er sett upp er ekki DAG (stefnt óhringað net) en það má breyta því í DAG með því að taka út hnútana sem samsvara stoppistöðvum og bæta við hnút fyrir stoppistöð og hvern tímapunkt (x, t), þá setjum við legg frá (x, t) til (x, t′) ef t′ er næsti tímapunktur fyrir þessa stöppistöð og legg sem fara frá (x, t) til v ef vagn fer frá x á tíma t.

Finnið, með einhverju móti, lengstu mögulegu leið sem tekur sem flesta ólíka strætis-
vagnaleiðir.